In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dense, Input, Flatten, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D,GlobalMaxPooling1D,GlobalMaxPooling2D
from keras.layers import Reshape, Dropout, Concatenate, LSTM,Bidirectional,BatchNormalization
from keras.layers import Flatten,Activation,CuDNNGRU,CuDNNLSTM
from keras.models import Model
from keras.models import Sequential
from keras import regularizers,optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
import os
import json
from IPython.core.display import display, HTML
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


Using TensorFlow backend.


In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [0]:
PAD_ROWS=26 
PAD_COLS=1800

In [0]:
print(os.listdir('drive/My Drive/DL Dataset'))
dir = 'drive/My Drive/DL Dataset/'
f = open(os.path.join(dir, 'train.json'), encoding="utf8")
data = json.load(f)

['dev.json', 'train.json', 'valence_1000.h5']


In [0]:
df = pd.DataFrame(data)
df= df.T
TOTAL_DATA=df.shape[0]
df.head()

,valence,activation,features
0,0,1,"[[5.502810676891276, 5.389630715979907, 5.8907..."
1,1,1,"[[5.059076172970736, 5.288492317702101, 4.2633..."
2,0,1,"[[4.218546271669202, 4.961436495859291, 3.6650..."
3,1,0,"[[4.650364321573866, 4.523905028353254, 5.0168..."
4,0,1,"[[3.900221957277269, 2.7325726489808124, 2.565..."


In [0]:
df['labels'] = df[['valence', 'activation']].values.tolist()
df.head()

,valence,activation,features,labels
0,0,1,"[[5.502810676891276, 5.389630715979907, 5.8907...","[0, 1]"
1,1,1,"[[5.059076172970736, 5.288492317702101, 4.2633...","[1, 1]"
2,0,1,"[[4.218546271669202, 4.961436495859291, 3.6650...","[0, 1]"
3,1,0,"[[4.650364321573866, 4.523905028353254, 5.0168...","[1, 0]"
4,0,1,"[[3.900221957277269, 2.7325726489808124, 2.565...","[0, 1]"


In [0]:
feat = np.zeros((TOTAL_DATA,PAD_ROWS,PAD_COLS),dtype=np.float32)

feature_set=pd.DataFrame(np.zeros((TOTAL_DATA,2),dtype=np.float32),columns=['features','label'])
feature_set['features']=df['features']
code=0
for i in tqdm_notebook(range(TOTAL_DATA)):
    
    [r,c]=np.array(df['features'][i]).T.shape
    feat[i,:r,:c]=np.array((df['features'][i])).T  # For input to lstm : samples, time ,features
    feature_set['features'][i]=feat[i]
    v=df['valence'][i]
    a=df['activation'][i]
    
    if v==0 and a==0:
        code=0
    elif v==0 and a==1:
        code=1
    elif v==1 and a==0:
        code=2
    elif v==1 and a==1:
        code=3
    feature_set['label'][i]=code

<h4> Create Separate Labelset for **Valence** and **Activation** </h4>


In [0]:
labels=feature_set['label'].astype(int)
labels = pd.get_dummies(feature_set['label']).values.tolist()

In [0]:
X = feat
Y = np.array(labels)
print(Y)

[[0 1 0 0]
 [0 0 0 1]
 [0 1 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]]


In [0]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=96)
input_dim=X_train[0].shape

In [0]:
X_train.shape

(7020, 26, 1800)

In [0]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7020, 26, 1800)
(780, 26, 1800)
(7020, 4)
(780, 4)


In [0]:
# #CNN + LSTM (45.25% : 212 epochs)

# model = Sequential()
# model.add(Conv1D(filters=128, kernel_size=9, activation='relu', padding='same',input_shape=input_dim,name='Conv1'))
# model.add(Conv1D(filters=128, kernel_size=6, activation='relu',padding='same',name='Conv2'))
# model.add(CuDNNLSTM(1024, return_sequences=False,input_shape=input_dim,name="LSTM1"))
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu',name='Dense1'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(24, activation='relu',name='Dense2'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(4, activation='softmax'))

# model = Sequential()

# model.add(Conv1D(filters=128, kernel_size=9, activation='relu', input_shape=input_dim))
# model.add(BatchNormalization())
# model.add(Conv1D(filters=64, kernel_size=6, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Conv1D(filters=64, kernel_size=6, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model.add(Dropout(0.4))
# model.add(CuDNNGRU(32))


# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='sigmoid',name='Output'))


model = Sequential()
model.add(Conv1D(filters=128, kernel_size=9, activation='relu', padding='same',input_shape=input_dim,name='Conv1'))
model.add(Conv1D(filters=128, kernel_size=6, activation='relu',padding='same',name='Conv2'))
model.add(CuDNNLSTM(1024, return_sequences=False,name="LSTM1"))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu',name='Dense1'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(24, activation='relu',name='Dense2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(4, activation='softmax'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (CuDNNLSTM)            (None, 1024)              11575296  
_________________________________________________________________
Dense1 (Dense)               (None, 128)               131200    
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)               512       
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
Dense2 (Dense)               (None, 128)               16512     
_________________________________________________________________
batch_normalization_12 (Batc (None, 128)               512       
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)              

If validation loss >> training loss : overfitting.

If validation loss  > training loss : some overfitting.

If validation loss  < training loss: some underfitting.

If validation loss << training loss : underfitting.

In [0]:
warnings.filterwarnings("ignore")
model = Sequential()

model.add(CuDNNLSTM(1024, return_sequences=False,input_shape=input_dim,name="LSTM1"))
model.add(Dense(128,activation='relu',name='Dense1'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu',name='Dense2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu',name='Dense3'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu',name='Dense4'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax',name='Output'))

In [0]:
warnings.filterwarnings("ignore")
model = Sequential()

model.add(CuDNNGRU(1024, return_sequences=True,input_shape=input_dim,name="LSTM1"))
model.add(Dropout(0.2))
model.add(CuDNNGRU(1024, return_sequences=True,name="LSTM2"))
model.add(Dropout(0.2))
model.add(CuDNNGRU(512, return_sequences=False,name="LSTM3"))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu',name='Dense1'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax',name='Output'))

In [0]:

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['categorical_accuracy'])


In [0]:
warnings.filterwarnings("ignore")
model.fit(X_train, y_train, verbose=1, epochs=300,validation_data=(X_test, y_test), shuffle=True)  # train the model


Train on 7020 samples, validate on 780 samples
Epoch 1/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.3146 - categorical_accuracy: 0.3934 - val_loss: 1.2881 - val_categorical_accuracy: 0.3577
Epoch 2/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.2652 - categorical_accuracy: 0.4057 - val_loss: 1.2528 - val_categorical_accuracy: 0.3949
Epoch 3/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.2450 - categorical_accuracy: 0.4080 - val_loss: 1.2454 - val_categorical_accuracy: 0.4167
Epoch 4/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.2176 - categorical_accuracy: 0.4259 - val_loss: 1.2399 - val_categorical_accuracy: 0.4167
Epoch 5/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.2080 - categorical_accuracy: 0.4342 - val_loss: 1.2175 - val_categorical_accuracy: 0.4321
Epoch 6/300
7020/7020 [==============================] - 10s 1ms/step - loss: 1.1961 - categorical_accu

In [0]:
model.fit(X_train, y_train, verbose=1, epochs=20,validation_data=(X_test, y_test), shuffle=True)  # train the model

In [0]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (CuDNNLSTM)            (None, 1024)              11575296  
_________________________________________________________________
Dense1 (Dense)               (None, 128)               131200    
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)               512       
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
Dense2 (Dense)               (None, 128)               16512     
_________________________________________________________________
batch_normalization_12 (Batc (None, 128)               512       
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)              

In [0]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)
y_label = np.argmax(y_test, axis=1)
accuracy_score(y_label.tolist(), pred.tolist())


In [0]:
#pred = model.predict(X_train)
# pred= np.array((pred>0.5).tolist()).astype(int).flatten()
# accuracy_score(y_test.tolist(), pred.tolist())

In [0]:
pred

In [0]:
from keras.models import load_model

In [0]:
model.save('drive/My Drive/DL Dataset/valence_1000.h5')

In [0]:
model1 = Sequential()

model1.add(Conv1D(filters=128, kernel_size=9, activation='relu', input_shape=input_dim))
model1.add(BatchNormalization())
model1.add(Conv1D(filters=64, kernel_size=6, activation='relu'))
model1.add(Dropout(0.4))
model1.add(Conv1D(filters=64, kernel_size=6, activation='relu'))
model1.add(Dropout(0.4))
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model1.add(Dropout(0.4))
model1.add(CuDNNGRU(32))


model1.add(Dense(512, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.4))
model1.add(Dense(512, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.4))
model1.add(Dense(128, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.4))
model1.add(Dense(1, activation='sigmoid',name='Output'))